In [12]:
import sqlite3
from math import sin, cos, sqrt, atan2, radians
import csv
import xlrd
import xlwt
import xlsxwriter
import pandas as pd
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

In [15]:
#access pfasdb
conn = sqlite3.connect('pfasdb.sqlite')
cur = conn.cursor()
choice = input('Upload File? (answer y or n):')
if choice =="y":
    #input file is tailored to a work project where we have a list of sites where we want
    #to find if there are nearby ground water wells. Input file has site ID, name, latitude, and longitude.
    inputfile = input('Enter filename:')
    #output file will list
    outputname = input('Enter Output File Name:')
    if outputname.endswith('.xlsx') is False:
        outputname = outputname+'.xlsx'
    writer = pd.ExcelWriter(outputname, engine='xlsxwriter')
    wb = xlrd.open_workbook(inputfile)
    sheet = wb.sheet_by_index(0)
    distance_threshold = input('Enter Distance Threshold (miles):')
    # approximate radius of earth in miles
    R = 3959.999
    for row in range(sheet.nrows):
        site_id = sheet.cell_value(row,0)
        site_name = sheet.cell_value(row,1)
        site_lat = sheet.cell_value(row,2)
        site_long = sheet.cell_value(row,3)
        # with open('well_output.csv', 'w', newline='') as csvfile:
        # csvwriter = csv.writer(csvfile, delimiter=',',
        #                         quotechar='|', quoting=csv.QUOTE_MINIMAL)
        # csvwriter.writerow(['globalid', 'lat', 'long', 'direction'])
        site_lat_r = radians(site_lat)
        site_long_r = radians(site_long)
        df = pd.DataFrame(columns = ['globalid', 'locationid', 'lat', 'long','direction','distance','chemical','qualifier','value','rl','date'])
        #will query all wells meeting criteria we are interested in and iterate through them and calculating distance from site coordinates
        for well in cur.execute('SELECT GlobalID, LocationID, Latitude, Longitude, Chemical, Qualifier, Value, ReportingLimit, Date FROM pfas_ca WHERE chemical IN ("PFOA", "PFOS", "PFBSA") AND matrix = "Liquid" AND Longitude IS NOT NULL AND Latitude IS NOT NULL'):
            globalid = well[0]
            locationid = well[1]
            if well[2]=="" or well[2]==None or well[3] == "" or well[3] == None:
                continue
            well_lat = float(well[2])
            well_long = float(well[3])
            #print (well_lat, well_long)
            well_lat_output = well[2]
            well_long_output = well[3]
            chemical = well[4]
            qualifier = well[5]
            value = well[6]
            rl = well[7]
            date = well[8]
            if qualifier == "ND" and rl !="":
                qualifier =="<"
                value = well[7]
            if qualifier =="<" and rl !="":
                value = well[7]
            date = datetime.strptime(date, '%m/%d/%Y')
            
            if site_lat > well_lat:
                north_south = "south"
            else:
                north_south  = "north"
            if site_long > well_long:
                east_west = "west"
            else:
                east_west = "east"
            direction = north_south + '-' + east_west
            # site_lat = radians(site_lat)
            # site_long = radians(site_long)
            well_lat_r = radians(well_lat)
            well_long_r = radians(well_long)
            dlon = well_long_r - site_long_r
            dlat = well_lat_r - site_lat_r
            a = sin(dlat / 2)**2 + cos(well_lat_r) * cos(site_lat_r) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            distance = R * c
            if distance <= float(distance_threshold):
                data = pd.DataFrame([[globalid, locationid, well_lat_output, well_long_output, direction,distance,chemical, qualifier, value, rl, date]], columns = ['globalid', 'locationid', 'lat', 'long','direction','distance','chemical','qualifier','value','rl','date'])
                df = df.append(data)
            else:
                continue
        df.to_excel (writer,sheet_name=site_id, index=False)
writer.save()
cur.close()
print('Search Complete')

Upload File? (answer y or n):y
Enter filename:test.xls
Enter Output File Name:test_output
Enter Distance Threshold (miles):4
Search Complete
